# Tests after system restart
This notebook contains simple tests to re-start using the system 2-7-2022.  Everything had been powered off with the AC broken, and on top of that I had been working on something else lately and so I hadn't used lwacr since it was last reprovisioned.

Goals:

1. Send a snapshot and plot it.

2. Run a short threshold scan.

Most of this code is adapted from threshold scan notebook.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import casperfpga
import time
from lwa_f import snap2_fengine
#from lwautils import lwa_arx 
import pandas as pd
from cr_functions import *
import struct
import math
from scipy import special as sp
from scipy import optimize as oz
from scipy import stats as st

In [2]:
fpgfile="/home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2021-10-04_1837.fpg"
brdname='snap01'
#load firmware and initialize etc
brd=snap2_fengine.Snap2Fengine(brdname)
brd.program(fpgfile)

casperbrd = casperfpga.CasperFpga(brdname, transport=casperfpga.TapcpTransport)

2022-02-09 21:48:46,601 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2022-02-09 21:48:46,822 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-02-09 21:48:47,043 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
2022-02-09 21:48:47,056 - lwa_f.snap2_fengine:snap01 - INFO - Loading firmware /home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2021-10-04_1837.fpg to snap01
2022-02-09 21:49:08,417 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2022-02-09 21:49:08,639 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-02-09 21:49:08,852 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1


## Check which boards are connected

In [3]:
#Check that I can connect to any board (not trying snap07 because I know it's down)
for name in ["snap01", "snap02","snap03","snap04","snap05","snap06","snap08","snap09","snap10"]:
    print( name)
    bd=casperfpga.CasperFpga(name, transport=casperfpga.TapcpTransport)
    print(bd.listdev())
    print(bd.is_running())

snap01
['aaa_onegbe_one_GbE', 'adc_sync', 'ads5296_controller0_0', 'ads5296_controller0_1', 'ads5296_controller1_0', 'ads5296_controller1_1', 'ads5296_hardware_rst0', 'ads5296_hardware_rst1', 'ads5296_spi_controller0', 'ads5296_spi_controller1', 'ads5296_wb_ram0_0_0', 'ads5296_wb_ram0_0_1', 'ads5296_wb_ram0_0_2', 'ads5296_wb_ram0_0_3', 'ads5296_wb_ram0_1_0', 'ads5296_wb_ram0_1_1', 'ads5296_wb_ram0_1_2', 'ads5296_wb_ram0_1_3', 'ads5296_wb_ram1_0_0', 'ads5296_wb_ram1_0_1', 'ads5296_wb_ram1_0_2', 'ads5296_wb_ram1_0_3', 'ads5296_wb_ram1_1_0', 'ads5296_wb_ram1_1_1', 'ads5296_wb_ram1_1_2', 'ads5296_wb_ram1_1_3', 'autocorr_acc_cnt', 'autocorr_acc_len', 'autocorr_common_dout0_bram', 'autocorr_common_dout1_bram', 'autocorr_common_dout2_bram', 'autocorr_common_dout3_bram', 'autocorr_common_dout4_bram', 'autocorr_common_dout5_bram', 'autocorr_common_dout6_bram', 'autocorr_common_dout7_bram', 'autocorr_mux_sel', 'autocorr_trig', 'chan_reorder_dynamic_map1', 'corr_0_acc_cnt', 'corr_0_acc_len', 'cor

## Initialize and Set up data source

In [3]:
setup_data_source(casperbrd,"adc")
brd.initialize(read_only=False)
brd.sync.arm_sync()
brd.sync.sw_sync()

Data source set to adc
2022-02-09 21:49:30,796 - lwa_f.snap2_fengine:snap01 - INFO - Initializing block (writable): fpga
2022-02-09 21:49:30,797 - lwa_f.snap2_fengine:snap01 - INFO - Initializing block (writable): adc
2022-02-09 21:49:30,801 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0: Setting clock source to 1
2022-02-09 21:49:30,808 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 1: Setting clock source to 1
2022-02-09 21:49:30,813 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 0: Setting clock source to 1
2022-02-09 21:49:30,819 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 1: Setting clock source to 1
2022-02-09 21:49:40,004 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0 clock rates: [490656980, 98184645, 98233573, 0, 0]
2022-02-09 21:49:40,005 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0: MMCM locked
2022-02-09 21:49:41,016 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 1 clock rates: [490580917, 98145445, 98216924

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-02-09 21:49:51,815 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane delays:
[[408, 400, 408, 392, 392, 392, 392, 392], [376, 360, 360, 352, 360, 360, 352, 352], [344, 344, 336, 336, 344, 344, 344, 352], [376, 392, 376, 368, 376, 376, 376, 376], [360, 352, 352, 360, 360, 352, 352, 360], [328, 328, 336, 328, 328, 320, 320, 328], [344, 328, 328, 328, 336, 336, 328, 328], [368, 368, 368, 368, 368, 376, 384, 360]]
2022-02-09 21:49:51,816 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane slacks:
[[40, 40, 40, 40, 32, 32, 40, 32], [40, 40, 40, 40, 48, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 32], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 32, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40]]
2022-02-09 21:49:51,817 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXX
2022-02-09 21:49:51,817 - lwa_f.blocks.block:snap01 - INF

2022-02-09 21:49:51,857 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|-----XXXXXXXXXXXXXXXXX
2022-02-09 21:49:51,857 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|-----XXXXXXXXXXXXXXXXX
2022-02-09 21:49:51,858 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXX
2022-02-09 21:49:51,859 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXX
2022-02-09 21:49:51,859 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXX
2022-02-09 21:49:51,860 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|-----XXXXXXXXXXXXXXXXX
2022-02-09 21:49:51,861 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 7, Lane 0:    XXXXXXXX

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


...

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-02-09 21:51:17,019 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 0 because nowhere was good
2022-02-09 21:51:17,031 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 1 because nowhere was good
2022-02-09 21:51:17,041 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 2 because nowhere was good
2022-02-09 21:51:17,053 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 3 because nowhere was good
2022-02-09 21:51:17,963 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 Scanning data delays


100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-02-09 21:51:25,045 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane delays:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [280, 280, 280, 296, 280, 280, 280, 280], [216, 216, 216, 216, 216, 216, 224, 216], [272, 256, 264, 280, 264, 264, 264, 264], [304, 304, 304, 296, 296, 296, 296, 296]]
2022-02-09 21:51:25,046 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane slacks:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [40, 40, 40, 40, 40, 40, 40, 40], [32, 32, 32, 32, 32, 32, 40, 32], [40, 32, 40, 40, 32, 40, 40, 32], [40, 40, 40, 40, 40, 40, 40, 40]]
2022-02-09 21:51:25,047 - lwa_f.blocks.block:snap01 - WARNING - adc - Delay solutions have small slack
2022-02-09 21:51:25,048 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-02-09 21:51:25,048 - lwa_f.blocks.block:snap0

2022-02-09 21:51:25,075 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 1:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-02-09 21:51:25,076 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-02-09 21:51:25,076 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXX
2022-02-09 21:51:25,077 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-02-09 21:51:25,077 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-02-09 21:51:25,078 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-02-09 21:51:25,079 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXX

## Send a snapshot
Don't forget to start and stop the datacapture script separately.

In [4]:
#send a snapshot just in case
setup_ethernet(brdname,casperbrd,fpgfile,'lwacr',100)
#enable ethernet
setvalue(casperbrd,'eth_enable','cr_registers.xlsx',1)
#send trigger
software_trigger(casperbrd,0)

------------------------
snap01:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:01:01:01
Gateway:  0.0.0.1
IP:  10.41.0.201
Fabric port: 
11111
Fabric interface is currently: Enabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips: []
ARP Table: 
IP: 10.41.0. 58: MAC: 04:3F:72:DF:C2:F8
Setting wait between packets to 100


In [32]:
val=getvalue(casperbrd,'readout_state_timer','cr_registers.xlsx')
print(val)

692934144


In [33]:
reset_to_listen(casperbrd)
print(getvalue(casperbrd,"readout_state_value","cr_registers.xlsx"))

0


In [34]:
print(getvalue(casperbrd,"readout_state_value","cr_registers.xlsx"))

0


In [35]:
previous_packets = getvalue(casperbrd,'eof_and_valid','cr_registers.xlsx')
previous_afull = getvalue(casperbrd,'almostfull','cr_registers.xlsx')
previous_overflows = getvalue(casperbrd,'overflow','cr_registers.xlsx')
print(previous_packets,previous_afull,previous_overflows)

3386048 5 0


In [36]:
print(getvalue(casperbrd,'readout_state_value','cr_registers.xlsx'))

0


In [37]:
setvalue(casperbrd,"send_trigger","cr_registers.xlsx",0)
setvalue(casperbrd,"send_trigger","cr_registers.xlsx",1)
packets = getvalue(casperbrd,'eof_and_valid','cr_registers.xlsx')
print(packets)

val = getvalue(casperbrd,'readout_state_value','cr_registers.xlsx')
if val == 0:
    print("Board successfully returned to listening state after readout")
if val !=0:
    print("Board did not return to listening state after readout. State value = " + str(val))
#Read register readout_state_timer.
val = getvalue(casperbrd,'readout_state_timer','cr_registers.xlsx')
print ("Readout took " + str(val) + " clock cycles")

#Read overflow counter to confirm that the 40Gbe block did not overflow.
of = getvalue(casperbrd,'overflow','cr_registers.xlsx')
overflows = of - previous_overflows
print(overflows)

3398400
Board successfully returned to listening state after readout
Readout took 700999488 clock cycles
0


## Plot the snapshot


In [ ]:
snapshotpath = ''#fill in after capture
snapshot=np.load(snapshotpath)

#plot spectra
fbins=np.linspace(0,197/2,int(1+4096/2))

plt.figure(figsize=(20,15))
for i in range(64):
    plt.subplot(8,8,1+i)
    plt.title(i)
    spec=np.fft.rfft(snapshot[:,i+4])
    plt.plot(fbins,np.log(np.square(np.abs(spec))))
    plt.ylim(0,25)
    if i > 55:
        plt.xlabel('frequency [MHz]')
    if i%8==0:
        plt.ylabel('power')

## Do a threshold scan

In [11]:
thresholds = 3*np.asarray(range(100))
thresholds = 3*np.asarray(range(100))
vthresholds = np.flip(thresholds)
results_core = np.zeros((64,100))
results_veto = np.zeros((64,100))

for i in range(len(thresholds)):
    t_c = thresholds[i]
    t_v = vthresholds[i]
    setvalue(casperbrd,"trigger_power_thresh","cr_registers.xlsx",int(t_c**2))
    setvalue(casperbrd,"veto_power_thresh","cr_registers.xlsx",int(t_v**2))
    time.sleep(2.75)  #have to wait for it to update
    core,veto = read_threshold_rates(casperbrd)
    results_core[:,i] = core
    results_veto[:,i] = veto

In [ ]:
n=272022
save the arrays
np.save("sky_thresholds_core"+str(n),thresholds)
np.save("sky_thresholds_veto"+str(n),vthresholds)
np.save("sky_results_core"+str(n),results_core)
np.save("sky_results_veto"+str(n),results_veto)

## Plot threshold scan results


In [ ]:
n=272022
savedthresholds=np.load("sky_thresholds_core"+str(n)+".npy")
savedvthresholds=np.load("sky_thresholds_veto"+str(n)+".npy")
savedresults_core=np.load("sky_results_core"+str(n)+".npy")
savedresults_veto=np.load("sky_results_veto"+str(n)+".npy")

In [ ]:
rate_core = savedresults_core/(2**28 - 1)
rate_veto = savedresults_veto/(2**28 - 1)

print("Fit sigma only")
popt1,pcov1 = oz.curve_fit(prob_simple,thresholds,rate_core[1,:])
print(popt1)
print(pcov1)

print("Fit Sigma and Mean")
p0=(0,10)
popt2,pcov2 = oz.curve_fit(prob,thresholds,rate_core[1,:],p0)
print(popt2)
print(pcov2)

model1 = prob_simple(thresholds,popt1[0])
model2 = prob(thresholds,popt2[0],popt2[1])

plt.figure(figsize=(10,10))
plt.plot(savedthresholds,rate_core[1,:],'.',label='core')
plt.plot(savedthresholds,model1,label="Core Fit sigma only")
plt.plot(savedthresholds,model2,label="Core Fit sigma and mean")


plt.plot(np.flip(savedthresholds),rate_veto[1,:],'.',label='veto')
plt.legend()

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(savedresults_core[1,:]-np.flip(savedresults_veto[1,:]),'C3',label="Core - veto")
plt.plot(binomial_error(rate_core[1,:],2**28),'C4',label="Estimate of uncertainty")
plt.ylabel("Core counts minus veto counts for same thresholds")
plt.xlabel("Threshold")

plt.subplot(122)
plt.plot(binomial_error(rate_core[1,:],2**28),'C4',label="Estimate of uncertainty")
plt.ylabel("Core counts minus veto counts for same thresholds")
plt.xlabel("Threshold")